## Continued pretraining for Llama2 70b SageMaker using AWS Trainium instances

This example provides a walkthrough of continued pretraining for [Llama 70b](https://huggingface.co/meta-llama/Llama-2-70b-hf) model using [NeuronX](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/transformers-neuronx/transformers-neuronx-developer-guide.html) Distributed on trainium instances. NeuronX Distributed is a package for supporting different distributed training/inference mechanism for Neuron devices. It would provide xla friendly implementations of some of the more popular distributed training/inference techniques. The library can be easily installed via pip.

First, we will obtain the path of the `pre-trained weights` from `Notebook 1`. These pre-trained weights represent a checkpoint or a starting point for the continual pretraining process. Continued-pretraining involves taking a pre-trained model (in this case, the pre-trained weights) and further pre-training it on additional to improve its  knowledge, capabilities, generalization across tasks and specific domains.

In this notebook, we showcase continued-pretraining for a Llama2 70B model by using the tensor parallel, pipeline parallel, sequence parallel, activation checkpoint as well as constant mask optimization in the neuronx-distributed package. 

### Contents

The example has the following main sections:

- [Install require packages](#Install-required-packages)
- [Download tokenizer and model](#Download-tokenizer-and-model)
- [Download training dataset](#Download-training-dataset)
- [Tokenize the data using Llama2 tokenizer](#Tokenize-the-data-using-Llama2-tokenizer)
- [Upload data to S3](#Upload-data-to-S3)
- [Run the training job](#Run-the-training-job)
- [Terminate the warmpool](#Terminate-the-warmpool)

### Instance type quota increase

Complete the following steps:

- Open the [Service Quotas console](https://console.aws.amazon.com/servicequotas/).
- Choose Amazon SageMaker.
- Choose the service quota.
- Choose Request quota increase.

**Notes**: *To make sure that you have enough quotas to support your usage requirements, it's a best practice to monitor and manage your service quotas. Requests for Amazon EC2 service quota increases are subject to review by AWS engineering teams. Also, service quota increase requests aren't immediately processed when you submit a request. After your request is processed, you receive an email notification.*

*This Jupyter Notebook can be run on a t3.medium instance (`ml.t3.medium`). However, to fine-tune the `Llama 2 70B` model, we use the `trn1.32xlarge` instance type. Pre-training Llama 2 70b model can run in a `cluster of 8 trn1.32xlarge instances`.* However, we recommend running the training job in a cluster of `32` instances.  

*Before you run this notebook, you'll need to request a `quota increase of 32` from Amazon SageMaker for the following resources:*

1. *ml.trn1.32xlarge instance type for training job usage*
2. *ml.trn1.32xlarge instance type for training warm pool usage*
3. *Maximum number of instances per training job*

### Install required packages

In [ ]:
!pip install -U sagemaker boto3 --quiet
!pip install transformers datasets[s3] --quiet

### Download tokenizer and model

Update the [access token](https://huggingface.co/docs/hub/en/security-tokens) to download the tokenizer

In [ ]:
from huggingface_hub.hf_api import HfFolder
access_token = "hf_xxxx"
HfFolder.save_token(access_token)

In [ ]:
from transformers import AutoTokenizer
tokenizer_name = "meta-llama/Llama-2-70b-hf"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
block_size = 4096

### Download training dataset

For this example we will download the [wikicorpus data](https://huggingface.co/datasets/wikicorpus) from huggingface datasets.

In [ ]:
from datasets import load_dataset
from itertools import chain
import os

# Dataset
dataset_name = "wikicorpus"
dataset_config_name = "raw_en"

# Create cache directory
save_path = "data/wikicorpus_llama2_7B_tokenized_4k"
save_path = os.path.expanduser(save_path)

if not os.path.exists(save_path):
    os.makedirs(save_path)

# Download wikicorpus data
raw_datasets = load_dataset(
    dataset_name,
    dataset_config_name,
    cache_dir=save_path,
    trust_remote_code=True
)

column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

### Tokenize the data using Llama2 tokenizer

Tokenize training dataset with llama2 tokenizer and then upload it to S3 to use during training.

In [ ]:
# Tokenize training dataset
def tokenize_function(examples):
    return tokenizer(examples[text_column_name])


tokenized_datasets = raw_datasets.map(
    tokenize_function,
    batched=True,
    remove_columns=column_names,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)

if block_size > tokenizer.model_max_length:
    print("block_size > tokenizer.model_max_length")
block_size = min(block_size, tokenizer.model_max_length)


# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, and if the total_length < block_size  we exclude this batch and return an empty dict.
    # We could add padding if the model supported it instead of this drop, you can customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result


lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    load_from_cache_file=True,
    desc=f"Grouping texts in chunks of {block_size}",
)

# Final training dataset
train_dataset = lm_datasets["train"]
print(len(train_dataset))

### Upload data to S3

In [ ]:
import sagemaker
import boto3

sess = sagemaker.Session()

# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket = None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
region_name = sess.boto_region_name

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {region_name}")

In [ ]:
# save training dataset to s3
training_input_path = f's3://{sess.default_bucket()}/neuronx_distributed/data'
print(f"uploading training dataset to: {training_input_path}")
train_dataset.save_to_disk(training_input_path)

print(f"uploaded data to: {training_input_path}")

At this point we should have the data uploaded to S3 and ready to kick start the training job.

### Run the training job

For the training job we will be using Trn1.32xlarge instances. Each Trn1.32xlarge instances will have 32 neuron cores and we will use Tensor parallelism and pipeline parallelism to shard the model across neuron cores and train. The below cell provides basic setting for pretraining llama 2 70b using Trn1. 

*Note: Change the number of instances within the cluster to increase or decrease the job execution time. You can use down to `8 Trn1.32xlarge instances`*.

In [ ]:
PROCESSES_PER_NODE = 32

# Number of instances within the cluster
WORLD_SIZE = 32 # This is the number of nodes in cluster, change this if you want to tweak the instance_count parameter

# Global batch size
GBS = 512

# Input sequence length
SEQ_LEN = 4096

# Pipeline parallel degree
PP_DEGREE = 8

# Tensor parallel degree
TP_DEGREE = 8

# Data paralell size
DP = ((PROCESSES_PER_NODE * WORLD_SIZE / TP_DEGREE / PP_DEGREE))

# Batch size per model replica
BS = ((GBS / DP))

# Number microbatches for pipeline execution
# Setting same as BS so each microbatch contains a single datasample
NUM_MICROBATCHES = BS

# Number of total steps for which to train model.
MAX_STEPS = 1500 # This number should be adjusted to the step number when the loss function is approaching convergence.

# Timeout in seconds for training
MAX_RUN = 2 * (24 * 60 * 60) # After this amount of time Amazon SageMaker terminates the job regardless of its current status.

These hyperparameters for continually pre-training Llama2 70B model are similar to the ones for pretraining

In [ ]:
hyperparameters = {}
hyperparameters["train_batch_size"] = int(BS)
hyperparameters["use_meta_device_init"] = 1
hyperparameters["training_dir"] = "/opt/ml/input/data/train" # path where sagemaker uploads the training data
hyperparameters["training_config"] = "config.json" # config file containing llama 70b configuration , change this for tweaking the number of parameters.
hyperparameters["max_steps"] = MAX_STEPS
hyperparameters["seq_len"] = SEQ_LEN
hyperparameters["pipeline_parallel_size"] = PP_DEGREE
hyperparameters["tensor_parallel_size"] = TP_DEGREE
hyperparameters["num_microbatches"] = int(NUM_MICROBATCHES)
hyperparameters["lr"] = 0.00015
hyperparameters["min_lr"] = 1e-05
hyperparameters["beta1"] = 0.9
hyperparameters["beta2"] = 0.95
hyperparameters["weight_decay"] = 0.1
hyperparameters["warmup_steps"] = 2000
hyperparameters["constant_steps"] = 0
hyperparameters["use_zero1_optimizer"] = 1
hyperparameters["tb_dir"] = "/opt/ml/checkpoints/tensorboard" # The tensorboard logs will be stored here and eventually pushed to S3.

Hyperparameters for continually pre-training Llama2 70B model

In [ ]:
hyperparameters["checkpoint_dir"] = "/opt/ml/checkpoints/checkpts"
hyperparameters["checkpoint_freq"] = 10
hyperparameters["num_kept_checkpoint"] = 1
hyperparameters["use_zero1_optimizer"] = 1
hyperparameters["save_load_xser"] = 0
hyperparameters["pretrained_weight_dir"] = "/opt/ml/checkpoints/llama70b_weights"

In [ ]:
# Docker image for training a models on AWS Trainium
docker_image = f"763104351884.dkr.ecr.{region_name}.amazonaws.com/pytorch-training-neuronx:1.13.1-neuronx-py310-sdk2.15.0-ubuntu20.04"

For more details about neron docker images:
- [AWS Neuron Deep Learning Containers](https://github.com/aws-neuron/deep-learning-containers/tree/main0)
- [Available Deep Learning Containers Images](https://github.com/aws/deep-learning-containers/blob/master/available_images.md)

In [ ]:
import time
# Define Training Job Name 
job_name = f'llama-neuron-nemo-{time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime())}'

# Define checkpoint directory that contains the weights and other relevant data for the trained model
checkpoint_s3_uri = "s3://" + sagemaker_session_bucket + "/nemo_llama_experiment"
checkpoint_dir = '/opt/ml/checkpoints'

In [ ]:
# Define neuron chache directory
cache_dir = "/opt/ml/checkpoints/neuron_cache"

In [ ]:
# Environment variables to be set for use during training job 
env = {}
env['FI_PROVIDER'] = 'efa'
env['NCCL_PROTO'] = 'simple'
env['FI_EFA_USE_DEVICE_RDMA'] = '1'
env['RDMAV_FORK_SAFE'] = '1'
env['FI_EFA_FORK_SAFE'] = '1'
env['NCCL_SOCKET_IFNAME'] = 'ens'
#env['XLA_USE_BF16']='1'
env['NEURON_FUSE_SOFTMAX'] = '1'
env['MALLOC_ARENA_MAX'] = '128'
env['XLA_DOWNCAST_BF16'] = '1'
env['NEURON_RT_ASYNC_EXEC_MAX_INFLIGHT_REQUESTS'] = '5'
env['NCCL_SOCKET_IFNAME'] = '^lo,docker'
env['NEURON_CC_FLAGS'] = "--model-type=transformer --distribution-strategy=llm-training --enable-saturate-infinity --cache_dir=" + cache_dir

[PyTorch estimator](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) for training a job on Amazon SageMaker:

In [ ]:
from sagemaker.pytorch import PyTorch

# Handle end-to-end Amazon SageMaker training and deployment tasks.
pt_estimator = PyTorch(
    entry_point='run_llama_nxd.py',
    source_dir='./scripts',
    instance_type="ml.trn1.32xlarge",
    image_uri=docker_image,
    instance_count=WORLD_SIZE,
    max_run=MAX_RUN,
    hyperparameters=hyperparameters,
    role=role,
    base_job_name=job_name,
    environment=env,
    input_mode="FastFile",
    disable_output_compression=True,
    keep_alive_period_in_seconds=600, # this is added to enable warm pool capability
    checkpoint_s3_uri=checkpoint_s3_uri,
    checkpoint_local_path=checkpoint_dir,
    distribution={"torch_distributed": {"enabled": True}} # enable torchrun 
)

In [ ]:
# Start training job
pt_estimator.fit({"train": training_input_path})

### Terminate the warmpool

Execute the below cell to terminate the warmpool if you no longer need it.

In [ ]:
sess.update_training_job(pt_estimator.latest_training_job.job_name, resource_config={"KeepAlivePeriodInSeconds":0})

In this example we looked at how to continually pre-trained Llama2 70b model using Amazon SageMaker training jobs on AWS Trainium instance. 